# Panel Regression: Random Slopes and Variance Components

## Introduction to Random Slopes

In the previous notebook, we explored **random intercepts** where each group has its own baseline level but shares common slopes. This notebook extends to **random slopes** where groups can have different slopes for specific variables.

### When to Use Random Slopes

Use random slopes when:
- Groups have different **trajectories** over time
- The effect of a predictor varies across groups
- You have sufficient data per group (typically 20+ observations)
- Theory suggests group heterogeneity in responses

### Examples

**Random Intercepts Only**:
- Stores have different baseline sales levels
- But price sensitivity is the same across all stores

**Random Intercepts + Random Slopes**:
- Patients start at different health baselines (random intercept)
- AND recover at different rates (random slope for time)

### Variance Components

Random slopes models estimate:
1. **Intercept variance**: Between-group variation in baselines
2. **Slope variance**: Between-group variation in slopes
3. **Covariance**: Correlation between intercepts and slopes
   - Positive: Groups with higher intercepts have steeper slopes
   - Negative: Groups with higher intercepts have flatter slopes

---

## Learning Objectives

In this notebook, you will:
1. Generate longitudinal patient clinical trial data
2. Fit random intercepts only model
3. Fit random intercepts + random slopes model
4. Extract and interpret variance components
5. Compare models using AIC/BIC
6. Visualize individual patient trajectories
7. Make group-specific predictions

## Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# py-tidymodels imports
from py_parsnip import panel_reg
from py_workflows import workflow

# Set random seed for reproducibility
np.random.seed(42)

# Plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

## 1. Generate Patient Clinical Trial Data

We'll create a realistic longitudinal dataset with:
- **20 patients**: Each measured at 6 time points
- **Total**: 120 observations
- **Variables**:
  - `health_score`: Patient health outcome (0-100 scale)
  - `time`: Months since treatment start (0-5)
  - `treatment`: Binary indicator (0 = control, 1 = treatment)
  - `patient_id`: Patient identifier (group)

**Data Generating Process**:
- Random intercepts: Patients start at 40-60 baseline health
- Random slopes: Recovery rates vary (0.5 to 2.5 points/month)
- Treatment effect: +5 points (same for all patients)
- Positive correlation between intercepts and slopes (sicker patients recover faster)

In [ ]:
# Parameters
n_patients = 20
n_timepoints = 6
n_total = n_patients * n_timepoints

# Random effects (correlated intercepts and slopes)
# Covariance matrix for random effects
mean_intercept = 50  # Average baseline health
mean_slope = 1.5     # Average recovery rate
var_intercept = 100  # Variance in baselines
var_slope = 0.5      # Variance in recovery rates
cov_int_slope = 3    # Positive covariance (sicker patients recover faster)

cov_matrix = np.array([
    [var_intercept, cov_int_slope],
    [cov_int_slope, var_slope]
])

# Generate correlated random effects
random_effects = np.random.multivariate_normal(
    mean=[mean_intercept, mean_slope],
    cov=cov_matrix,
    size=n_patients
)

random_intercepts = random_effects[:, 0]
random_slopes = random_effects[:, 1]

# Fixed effects
beta_treatment = 5.0  # Treatment increases health by 5 points

# Generate data
data_list = []

for i in range(n_patients):
    patient_id = f'Patient_{i+1:02d}'
    
    # Time points (0 to 5 months)
    time = np.arange(n_timepoints)
    
    # Treatment assignment (50% treated, 50% control)
    treatment = 1 if i % 2 == 0 else 0
    
    # Health score = intercept + slope*time + treatment*beta + noise
    health_score = (
        random_intercepts[i] + 
        random_slopes[i] * time + 
        beta_treatment * treatment + 
        np.random.normal(0, 3, n_timepoints)
    )
    
    patient_data = pd.DataFrame({
        'patient_id': patient_id,
        'time': time,
        'treatment': treatment,
        'health_score': health_score
    })
    
    data_list.append(patient_data)

# Combine all patients
clinical_data = pd.concat(data_list, ignore_index=True)

print(f"Dataset shape: {clinical_data.shape}")
print(f"\nFirst few rows:")
print(clinical_data.head(12))
print(f"\nSummary statistics:")
print(clinical_data.describe())
print(f"\nTreatment distribution:")
print(clinical_data.groupby('patient_id')['treatment'].first().value_counts())

In [ ]:
# Visualize individual patient trajectories
fig, ax = plt.subplots(figsize=(12, 7))

# Plot each patient's trajectory
for patient_id in clinical_data['patient_id'].unique():
    patient_data = clinical_data[clinical_data['patient_id'] == patient_id]
    treatment = patient_data['treatment'].iloc[0]
    
    color = 'steelblue' if treatment == 1 else 'coral'
    alpha = 0.6
    
    ax.plot(patient_data['time'], patient_data['health_score'], 
            'o-', color=color, alpha=alpha, linewidth=1.5, markersize=5)

# Create custom legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], color='steelblue', lw=2, label='Treatment'),
    Line2D([0], [0], color='coral', lw=2, label='Control')
]
ax.legend(handles=legend_elements, loc='upper left', fontsize=11)

ax.set_xlabel('Time (months)', fontsize=12)
ax.set_ylabel('Health Score', fontsize=12)
ax.set_title('Patient Health Trajectories Over Time', fontsize=14, weight='bold')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📊 Key Observations:")
print("   - Patients start at different baseline health levels (random intercepts)")
print("   - Patients recover at different rates (random slopes)")
print("   - Treatment group (blue) has higher overall health scores")

## 2. Fit Random Intercepts Only Model

First, let's fit a model with only random intercepts (no random slopes). This assumes all patients recover at the same rate, but start at different baselines.

In [ ]:
# Create panel regression specification (intercepts only)
spec_intercept = panel_reg(random_effects="intercept")

# Create workflow
wf_intercept = workflow().add_formula("health_score ~ time + treatment").add_model(spec_intercept)

# Fit using fit_global()
fit_intercept = wf_intercept.fit_global(clinical_data, group_col='patient_id')

print("✅ Random intercepts model fitted successfully!")
print(f"\nModel type: {fit_intercept.spec.model_type}")
print(f"Random effects: {fit_intercept.spec.args['random_effects']}")

In [ ]:
# Extract outputs
outputs_int, coefficients_int, stats_int = fit_intercept.extract_outputs()

print("\n" + "="*60)
print("RANDOM INTERCEPTS MODEL - COEFFICIENTS")
print("="*60)
print("\nFixed Effects:")
fixed_int = coefficients_int[coefficients_int['type'] == 'fixed'][['variable', 'coefficient', 'std_error', 'p_value']]
print(fixed_int.to_string(index=False))

print("\nVariance Components:")
var_components_int = coefficients_int[coefficients_int['type'].isin(['random', 'residual'])][['variable', 'coefficient', 'type']]
print(var_components_int.to_string(index=False))

In [ ]:
# Model statistics
rmse_int = stats_int[stats_int['metric'] == 'rmse']['value'].values[0]
r2_int = stats_int[stats_int['metric'] == 'r_squared']['value'].values[0]
aic_int = stats_int[stats_int['metric'] == 'aic']['value'].values[0]
bic_int = stats_int[stats_int['metric'] == 'bic']['value'].values[0]
icc_int = stats_int[stats_int['metric'] == 'icc']['value'].values[0]

print("\n" + "="*60)
print("RANDOM INTERCEPTS MODEL - PERFORMANCE")
print("="*60)
print(f"\nRMSE: {rmse_int:.3f}")
print(f"R²: {r2_int:.4f}")
print(f"AIC: {aic_int:.2f}")
print(f"BIC: {bic_int:.2f}")
print(f"ICC: {icc_int:.4f} ({icc_int*100:.1f}% variance between patients)")

## 3. Fit Random Intercepts + Random Slopes Model

Now let's fit a model with both random intercepts and random slopes. This allows each patient to have:
- Their own baseline health (random intercept)
- Their own recovery rate (random slope for `time`)

In [ ]:
# Create panel regression specification (intercepts + slopes)
spec_slopes = panel_reg(random_effects="both")

# Set slope variable (must specify which variable has random slope)
spec_slopes = spec_slopes.set_args(slope_var='time')

# Create workflow
wf_slopes = workflow().add_formula("health_score ~ time + treatment").add_model(spec_slopes)

# Fit using fit_global()
fit_slopes = wf_slopes.fit_global(clinical_data, group_col='patient_id')

print("✅ Random intercepts + slopes model fitted successfully!")
print(f"\nModel type: {fit_slopes.spec.model_type}")
print(f"Random effects: {fit_slopes.spec.args['random_effects']}")
print(f"Slope variable: {fit_slopes.spec.args['slope_var']}")

In [ ]:
# Extract outputs
outputs_slopes, coefficients_slopes, stats_slopes = fit_slopes.extract_outputs()

print("\n" + "="*60)
print("RANDOM INTERCEPTS + SLOPES MODEL - COEFFICIENTS")
print("="*60)
print("\nFixed Effects:")
fixed_slopes = coefficients_slopes[coefficients_slopes['type'] == 'fixed'][['variable', 'coefficient', 'std_error', 'p_value']]
print(fixed_slopes.to_string(index=False))

print("\nVariance Components:")
var_components_slopes = coefficients_slopes[coefficients_slopes['type'].isin(['random', 'residual'])][['variable', 'coefficient', 'type']]
print(var_components_slopes.to_string(index=False))

## 4. Extract and Interpret Variance Components

The random slopes model provides three variance components:
1. **RE: Intercept Variance**: How much patients vary in baseline health
2. **RE: time Variance**: How much patients vary in recovery rates
3. **RE: Cov(Intercept, time)**: Correlation between baseline and recovery rate

The covariance is particularly interesting:
- **Positive**: Patients with higher baselines recover faster
- **Negative**: Patients with higher baselines recover slower
- **Zero**: No relationship between baseline and recovery rate

In [ ]:
# Extract variance components
intercept_var = coefficients_slopes[coefficients_slopes['variable'] == 'RE: Intercept Variance']['coefficient'].values[0]
slope_var = coefficients_slopes[coefficients_slopes['variable'] == 'RE: time Variance']['coefficient'].values[0]
cov_int_slope_est = coefficients_slopes[coefficients_slopes['variable'] == 'RE: Cov(Intercept, time)']['coefficient'].values[0]
residual_var = coefficients_slopes[coefficients_slopes['variable'] == 'Residual Variance']['coefficient'].values[0]

# Calculate correlation from covariance
correlation_int_slope = cov_int_slope_est / np.sqrt(intercept_var * slope_var)

print("\n" + "="*60)
print("VARIANCE COMPONENTS INTERPRETATION")
print("="*60)
print(f"\n📊 Random Intercept Variance: {intercept_var:.3f}")
print(f"   → Standard deviation: {np.sqrt(intercept_var):.2f} health points")
print(f"   → 95% of patients have baseline health within ±{1.96*np.sqrt(intercept_var):.1f} of mean")

print(f"\n📈 Random Slope Variance: {slope_var:.3f}")
print(f"   → Standard deviation: {np.sqrt(slope_var):.2f} points/month")
print(f"   → 95% of patients have recovery rates within ±{1.96*np.sqrt(slope_var):.2f} points/month of mean")

print(f"\n🔗 Covariance(Intercept, Slope): {cov_int_slope_est:.3f}")
print(f"   → Correlation: {correlation_int_slope:.3f}")
if correlation_int_slope > 0.3:
    print(f"   → Positive correlation: Patients with HIGHER baseline health recover FASTER")
    print(f"   → (Healthier patients respond better to treatment)")
elif correlation_int_slope < -0.3:
    print(f"   → Negative correlation: Patients with LOWER baseline health recover FASTER")
    print(f"   → (Sicker patients have more room for improvement)")
else:
    print(f"   → Weak correlation: Baseline health and recovery rate are independent")

print(f"\n⚡ Residual Variance: {residual_var:.3f}")
print(f"   → Within-patient variation not explained by model")

In [ ]:
# Visualize variance components
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Variance components bar chart
variance_components = {
    'Intercept\nVariance': intercept_var,
    'Slope\nVariance': slope_var,
    'Residual\nVariance': residual_var
}

axes[0].bar(variance_components.keys(), variance_components.values(), 
            color=['steelblue', 'coral', 'lightgray'])
axes[0].set_ylabel('Variance', fontsize=12)
axes[0].set_title('Variance Components Breakdown', fontsize=13, weight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Plot 2: Correlation visualization
# Extract patient-specific random effects
random_effects_dict = fit_slopes.fit_data['random_effects']
patient_intercepts = [effects['Group'] for effects in random_effects_dict.values()]
patient_slopes = [effects['time'] for effects in random_effects_dict.values()]

axes[1].scatter(patient_intercepts, patient_slopes, s=100, alpha=0.6, color='darkgreen')
axes[1].set_xlabel('Random Intercept (Baseline Deviation)', fontsize=12)
axes[1].set_ylabel('Random Slope (Recovery Rate Deviation)', fontsize=12)
axes[1].set_title(f'Intercept vs Slope (r = {correlation_int_slope:.3f})', fontsize=13, weight='bold')
axes[1].axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
axes[1].axvline(x=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
axes[1].grid(True, alpha=0.3)

# Add regression line
z = np.polyfit(patient_intercepts, patient_slopes, 1)
p = np.poly1d(z)
axes[1].plot(patient_intercepts, p(patient_intercepts), "r--", linewidth=2, alpha=0.8, label='Trend')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\n🔍 The scatter plot shows the relationship between patient baselines and recovery rates.")
if correlation_int_slope > 0.3:
    print(f"   Positive trend: Healthier patients at baseline tend to recover faster.")
elif correlation_int_slope < -0.3:
    print(f"   Negative trend: Sicker patients at baseline tend to recover faster.")

## 5. Model Comparison: Intercepts vs Slopes

Let's compare the two models using:
- **AIC/BIC**: Lower is better (penalizes model complexity)
- **RMSE**: Root Mean Square Error (lower is better)
- **R²**: Proportion of variance explained (higher is better)

The random slopes model has more parameters, so AIC/BIC will penalize it. But if the improvement in fit is substantial, it will still have lower AIC/BIC.

In [ ]:
# Extract performance metrics for both models
rmse_slopes = stats_slopes[stats_slopes['metric'] == 'rmse']['value'].values[0]
r2_slopes = stats_slopes[stats_slopes['metric'] == 'r_squared']['value'].values[0]
aic_slopes = stats_slopes[stats_slopes['metric'] == 'aic']['value'].values[0]
bic_slopes = stats_slopes[stats_slopes['metric'] == 'bic']['value'].values[0]

# Create comparison table
comparison_df = pd.DataFrame({
    'Metric': ['RMSE', 'R²', 'AIC', 'BIC'],
    'Intercepts Only': [
        f"{rmse_int:.3f}",
        f"{r2_int:.4f}",
        f"{aic_int:.2f}",
        f"{bic_int:.2f}"
    ],
    'Intercepts + Slopes': [
        f"{rmse_slopes:.3f}",
        f"{r2_slopes:.4f}",
        f"{aic_slopes:.2f}",
        f"{bic_slopes:.2f}"
    ],
    'Better Model': [
        'Slopes' if rmse_slopes < rmse_int else 'Intercepts',
        'Slopes' if r2_slopes > r2_int else 'Intercepts',
        'Slopes' if aic_slopes < aic_int else 'Intercepts',
        'Slopes' if bic_slopes < bic_int else 'Intercepts'
    ]
})

print("\n" + "="*60)
print("MODEL COMPARISON")
print("="*60)
print("\n" + comparison_df.to_string(index=False))

# Determine best model
aic_improvement = aic_int - aic_slopes
bic_improvement = bic_int - bic_slopes

print(f"\n" + "="*60)
print("INTERPRETATION")
print("="*60)
print(f"\n💡 AIC Improvement: {aic_improvement:.2f} points")
print(f"   (Positive = slopes model is better)")
print(f"\n💡 BIC Improvement: {bic_improvement:.2f} points")
print(f"   (Positive = slopes model is better)")

if aic_slopes < aic_int and bic_slopes < bic_int:
    print(f"\n🏆 RECOMMENDATION: Use random intercepts + slopes model")
    print(f"   Both AIC and BIC favor the more complex model.")
    print(f"   The improvement in fit justifies the added complexity.")
elif aic_slopes < aic_int:
    print(f"\n⚠️ MIXED RESULTS: AIC favors slopes, BIC favors intercepts")
    print(f"   BIC penalizes complexity more heavily than AIC.")
    print(f"   Consider: (1) Sample size, (2) Theoretical justification for random slopes")
else:
    print(f"\n🏆 RECOMMENDATION: Use random intercepts only model")
    print(f"   The added complexity of random slopes is not justified.")
    print(f"   Patients have similar recovery rates.")

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: RMSE comparison
models = ['Intercepts\nOnly', 'Intercepts +\nSlopes']
rmse_values = [rmse_int, rmse_slopes]
colors_rmse = ['coral' if v == min(rmse_values) else 'lightgray' for v in rmse_values]

axes[0].bar(models, rmse_values, color=colors_rmse)
axes[0].set_ylabel('RMSE', fontsize=12)
axes[0].set_title('Model Comparison: RMSE (Lower is Better)', fontsize=13, weight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Add values on bars
for i, v in enumerate(rmse_values):
    axes[0].text(i, v + 0.1, f'{v:.3f}', ha='center', fontsize=11, weight='bold')

# Plot 2: AIC/BIC comparison
x = np.arange(len(models))
width = 0.35

aic_values = [aic_int, aic_slopes]
bic_values = [bic_int, bic_slopes]

axes[1].bar(x - width/2, aic_values, width, label='AIC', color='steelblue')
axes[1].bar(x + width/2, bic_values, width, label='BIC', color='darkgreen')
axes[1].set_ylabel('Information Criterion', fontsize=12)
axes[1].set_title('Model Comparison: AIC/BIC (Lower is Better)', fontsize=13, weight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels(models)
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 6. Visualize Individual Patient Trajectories

Let's compare how the two models fit individual patient trajectories. The random slopes model should capture patient-specific recovery rates better.

In [ ]:
# Plot 4 example patients with both model fits
example_patients = ['Patient_01', 'Patient_05', 'Patient_10', 'Patient_15']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for i, patient_id in enumerate(example_patients):
    # Get actual data
    patient_actual = clinical_data[clinical_data['patient_id'] == patient_id]
    
    # Get fitted values from both models
    patient_fit_int = outputs_int[outputs_int['group'] == patient_id]
    patient_fit_slopes = outputs_slopes[outputs_slopes['group'] == patient_id]
    
    # Plot actual data
    axes[i].scatter(patient_actual['time'], patient_actual['health_score'], 
                   s=100, color='black', label='Actual', zorder=3)
    
    # Plot fitted values
    axes[i].plot(patient_actual['time'], patient_fit_int['fitted'], 
                'o--', color='coral', label='Intercepts Only', linewidth=2, markersize=7)
    axes[i].plot(patient_actual['time'], patient_fit_slopes['fitted'], 
                's-', color='steelblue', label='Intercepts + Slopes', linewidth=2, markersize=7)
    
    axes[i].set_xlabel('Time (months)', fontsize=11)
    axes[i].set_ylabel('Health Score', fontsize=11)
    axes[i].set_title(patient_id, fontsize=12, weight='bold')
    axes[i].legend(fontsize=9)
    axes[i].grid(True, alpha=0.3)

plt.suptitle('Model Comparison: Individual Patient Trajectories', fontsize=14, weight='bold')
plt.tight_layout()
plt.show()

print("\n🎯 The random slopes model (blue) typically fits individual trajectories better")
print("   because it allows each patient to have their own recovery rate.")

## 7. Group-Specific Predictions

Let's extract patient-specific random effects and make predictions for:
1. An existing patient (uses patient-specific effects)
2. A new patient (uses population average effects)

In [ ]:
# Extract patient-specific random effects
random_effects_dict = fit_slopes.fit_data['random_effects']

patient_effects_list = []
for patient_id, effects in random_effects_dict.items():
    patient_effects_list.append({
        'patient_id': patient_id,
        'random_intercept': effects['Group'],
        'random_slope': effects['time']
    })

patient_effects_df = pd.DataFrame(patient_effects_list).sort_values('random_intercept', ascending=False)

print("\n" + "="*60)
print("PATIENT-SPECIFIC RANDOM EFFECTS")
print("="*60)
print("\nTop 10 patients (sorted by baseline deviation):")
print(patient_effects_df.head(10).to_string(index=False))

print("\n💡 Interpretation:")
print("   - Positive random intercept: Patient starts healthier than average")
print("   - Positive random slope: Patient recovers faster than average")

In [ ]:
# Make predictions for Patient_01 (existing) and Patient_NEW (new)
# Create future time points (6-11 months)
future_times = np.arange(6, 12)

# Existing patient (Patient_01 - treatment group)
patient_01_treatment = clinical_data[clinical_data['patient_id'] == 'Patient_01']['treatment'].iloc[0]
test_patient_01 = pd.DataFrame({
    'patient_id': ['Patient_01'] * len(future_times),
    'time': future_times,
    'treatment': patient_01_treatment
})

# New patient (assume treatment group)
test_patient_new = pd.DataFrame({
    'patient_id': ['Patient_NEW'] * len(future_times),
    'time': future_times,
    'treatment': 1
})

# Make predictions
preds_patient_01 = fit_slopes.predict(test_patient_01)
preds_patient_new = fit_slopes.predict(test_patient_new)

print("\n" + "="*60)
print("PREDICTIONS FOR FUTURE TIME POINTS")
print("="*60)
print("\nPatient_01 (existing patient):")
pred_df_01 = pd.DataFrame({
    'time': future_times,
    'predicted_health': preds_patient_01['.pred'].values
})
print(pred_df_01.to_string(index=False))

print("\nPatient_NEW (new patient):")
pred_df_new = pd.DataFrame({
    'time': future_times,
    'predicted_health': preds_patient_new['.pred'].values
})
print(pred_df_new.to_string(index=False))

In [ ]:
# Visualize predictions
fig, ax = plt.subplots(figsize=(12, 7))

# Historical data for Patient_01
patient_01_hist = clinical_data[clinical_data['patient_id'] == 'Patient_01']
ax.plot(patient_01_hist['time'], patient_01_hist['health_score'], 
        'o-', color='steelblue', linewidth=2, markersize=8, label='Patient_01 (historical)', zorder=3)

# Predictions for Patient_01
ax.plot(future_times, preds_patient_01['.pred'].values, 
        's--', color='steelblue', linewidth=2, markersize=8, label='Patient_01 (predicted)', alpha=0.7)

# Predictions for new patient
ax.plot(future_times, preds_patient_new['.pred'].values, 
        '^--', color='coral', linewidth=2, markersize=8, label='Patient_NEW (predicted)', alpha=0.7)

ax.axvline(x=5.5, color='black', linestyle=':', linewidth=2, alpha=0.5, label='Present')
ax.set_xlabel('Time (months)', fontsize=12)
ax.set_ylabel('Health Score', fontsize=12)
ax.set_title('Patient-Specific Predictions: Existing vs New Patient', fontsize=14, weight='bold')
ax.legend(fontsize=11, loc='upper left')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

mean_diff = (preds_patient_01['.pred'].values - preds_patient_new['.pred'].values).mean()
print(f"\n📊 Patient_01's predictions are {mean_diff:.2f} points {'higher' if mean_diff > 0 else 'lower'} than new patient.")
print(f"   This reflects Patient_01's specific random intercept and slope.")
print(f"\n   New patients get the 'population average' trajectory (no patient-specific adjustment).")

## Summary and Key Takeaways

### What We Learned

1. **Random slopes** extend random intercepts by allowing groups to have different slopes (not just baselines).

2. **When to use random slopes**:
   - Theory suggests group heterogeneity in effects
   - Sufficient data per group (20+ observations)
   - Visual inspection shows varying slopes across groups
   - AIC/BIC favor the more complex model

3. **Variance components** in random slopes models:
   - **Intercept variance**: Between-group variation in baselines
   - **Slope variance**: Between-group variation in slopes
   - **Covariance**: Correlation between intercepts and slopes
     - Positive: Higher intercepts → steeper slopes
     - Negative: Higher intercepts → flatter slopes

4. **Model selection** using AIC/BIC:
   - Lower AIC/BIC indicates better model
   - BIC penalizes complexity more than AIC
   - Improvement of 2-10 points: Some evidence
   - Improvement > 10 points: Strong evidence

5. **Covariance interpretation** is clinically meaningful:
   - Positive covariance: Healthier patients respond better
   - Negative covariance: Sicker patients have more room for improvement

6. **Patient-specific predictions** combine:
   - Fixed effects (population average)
   - Random effects (patient-specific deviations)

### Practical Guidelines

**Start with random intercepts**:
- Simpler model
- Easier to interpret
- Requires less data per group

**Add random slopes when**:
- Visual inspection shows varying slopes
- Theory supports heterogeneity
- Sufficient observations per group (20+)
- AIC/BIC improve substantially

**Interpret variance components carefully**:
- Focus on practical significance, not just statistical
- Covariance tells an important story about group behavior
- Standard deviations are more interpretable than variances

### Next Steps

In the next notebook, we'll explore:
- **Recipe preprocessing** with panel_reg()
- **WorkflowSet** for comparing multiple panel models
- **Multi-model comparison** across different specifications